## The Sentiment Analysis for Stabuck's Competitors

********************************************************************************************************************************
#Created at 26/07/2017
#Author: Ashley Hall
#Purpose: Analysing the sentiment of tweets for Starbucks competitors (Costa, McCafe and Dunkin Donuts) in multiple languages
********************************************************************************************************************************

In [ ]:
#importing relevant programs 
import pyodbc
from __future__ import absolute_import, print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from sqlalchemy.exc import ProgrammingError
import tweepy

#emoji allows emoticons to be picked up within the text
import emoji

#using vaderSentiment to conduct the analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math, re, string, requests, json

In [106]:
#connecting to DSN, created in SQL
conn = pyodbc.connect(r'DSN=costa')
curr= conn.cursor()
analyzer = SentimentIntensityAnalyzer()

#vader can only read certain languages, making the list to loop through language codes
language_codes = ["fr", "de", "es", "it", "ru", "ja", "ar", "zh"]

#the keys issued to my account to be able to connect to twitter
apikey = 'ZerLLgilL0c9olSlmhIwm01fl'
apisecret ='1kcyiPDk4jgCzgvZCarEWGaEHYLKnJcqfT52U7hvKNYe58EuCE'
accesstoken = '887695092588433415-EFIyXraH1bb3OQ7FqLxtKBwydNNjhE8'
tokensecret = 'Od2fLe0t6ppWkHsTmEUvto4Bh2QK2gJmsG9xZxejsSvNX'

In [109]:
class StdOutListener(StreamListener):
    def on_status(self, status):
        
        #try the below code
        try:
            
            #if loop to translate the tweet if the language of the tweet has a language code in the above list
            if status.user.lang in language_codes:
                api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(status.text, status.user.lang, "en")
                hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
                response = requests.get(api_url, headers=hdrs)
                response_json = json.loads(response.text)
                translation = response_json["responseData"]["translatedText"]
                translator_name = "MemoryNet Translation Service"
                
                #sentiment scores
                vs = analyzer.polarity_scores(translation)
                sent = vs['compound']
           
           #if tweet does not have a language code attached, then give the sentiment scores
            else:
                vs = analyzer.polarity_scores(status.text)
                sent = vs['compound']
                translation = status.text
                
        #if try doesn't work and there is an error, print the tweet here, rather than crashing, and move on.
        except:
            print(translation)
            
        #replacing apostrophes with two SQL can read them without thinking they are part of a string        
        status.text = str.replace(status.text, '\'', '\'\'')
        translation = str.replace(translation, '\'', '\'\'')
        
        #insert tweets into table in SQL
        q= 'insert into competitors(\
        id_str, created,Uk_time, text, fav,name, followers, sentiment, language, translated)\
        values (\'%s\',\'%s\', getdate(),\'%s\',\'%s\',\'%s\',\'%s\', \'%s\',\'%s\',\'%s\')'\
        %(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name,\
          status.user.followers_count, sent, status.user.lang, translation)
        
     
        try:
            curr.execute(q)
            conn.commit()
            
        except:
            print(q)
        #('insert into competitors(\
        #id_str, created,text, fav,name, description, loc, user_created, follower)\
        #values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'\
        #%(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name, status.user.description, status.user.location, status.user.created_at, status.user.followers_count))
        
        
     # if there is an error (locked out of twitter, print 420)    
               
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            return False

In [110]:
#connecting to twitter
l = StdOutListener()
auth = tweepy.OAuthHandler(apikey, apisecret)
auth.set_access_token(accesstoken, tokensecret)
api = tweepy.API(auth)
stream = Stream(auth, l)
#look for costa, mccafe and dunkin donuts tweets
stream.filter(track=['costacoffee','mccafe','dunkindonuts coffee'])

UnboundLocalError: local variable 'q' referenced before assignment